In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd "/content/drive/MyDrive"

/content/drive/MyDrive


In [ ]:
!pip install bitsandbytes
!pip install accelerate
!pip install transformers
!pip install datasets
!pip install evaluate

In [ ]:
from transformers import (
    ViTImageProcessor,
    ViTForImageClassification,
    TrainingArguments,
    Trainer,
    DefaultDataCollator,
)
from torchvision import transforms
from datasets import load_dataset
import numpy as np
import evaluate
import torch

PATH_TO_DATASET = "/content/drive/MyDrive/lung-cancer-detection/processedData"
MODEL_PATH = "google/vit-base-patch16-224"
# "google/vit-base-patch16-224"
# "cancer_model"


def get_dataset(folder_path):

    _label2id = {
        "cancerous": 0,
        "non-cancerous": 1,

    }
    _id2label = {
        0: "cancerous",
        1: "non-cancerous",

    }
    dataset = load_dataset("imagefolder", data_dir=folder_path)

    image_processor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224")

    def preprocess(samples):
        try:
            samples["pixel_values"] = image_processor.preprocess(
                images=samples["image"], return_tensors="pt"
            ).pixel_values
        except:
            samples["pixel_values"] = image_processor.preprocess(
                images=[img.convert("RGB") for img in samples["image"]],
                return_tensors="pt",
            ).pixel_values
        del samples["image"]
        return samples

    dataset = dataset.map(preprocess, batched=True, batch_size=3)
    print("Dataset loaded successfully.")
    return dataset, _id2label, _label2id, image_processor


def setup_model(base_model_path, _id2label, _label2id, _labels):
    model = ViTForImageClassification.from_pretrained(
        base_model_path,
        num_labels=len(_labels),
        id2label=_id2label,
        label2id=_label2id,
        ignore_mismatched_sizes=True,
    )

    print("Model loaded successfully.")
    return model


def compute_metrics(eval_pred):
    accuracy = evaluate.load("accuracy")
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)


def finetune_model(model, dataset, image_processor):
    data_collator = DefaultDataCollator()

    for param in model.base_model.parameters():
        param.requires_grad = False

    training_args = TrainingArguments(

        output_dir="cancer_trainer",
        remove_unused_columns=False,
        logging_strategy="epoch",
        evaluation_strategy="epoch",
        learning_rate=5e-3,
        optim="adamw_torch",
        save_strategy="epoch",
        per_device_train_batch_size=32,
        gradient_accumulation_steps=2,
        per_device_eval_batch_size=32,
        num_train_epochs=3,
        warmup_ratio=0.0,
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        weight_decay=0.00,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=dataset["train"],
        eval_dataset=dataset["test"],
        tokenizer=image_processor,
        compute_metrics=compute_metrics,
    )

    trainer.train()
    model.save_pretrained("cancer_model4")


def validate_model(model, test_dataset):
    model.to("cpu")
    model.eval()
    correct = [0, 0]
    incorrect = [0, 0]
    for sample in test_dataset:
        inputs = torch.as_tensor(sample["pixel_values"], dtype=torch.float32)
        inputs = inputs[None, :, :, :]
        with torch.no_grad():
            logits = model(inputs).logits
        predicted_label = logits.argmax(-1).item()
        if predicted_label == sample["label"]:
            correct[sample["label"]] += 1
        else:
            incorrect[sample["label"]] += 1
            print(
                "incorrect prediction... predicted:",
                model.config.id2label[predicted_label],
                "expected:",
                model.config.id2label[sample["label"]],
            )

    for i in range(len(correct)):
        print("\n", model.config.id2label[i])
        print("right:", correct[i], " wrong:", incorrect[i])
        print("accuracy:", correct[i] / (correct[i] + incorrect[i]))
    print("\noverall")
    print("right:", sum(correct), "wrong:", sum(incorrect))
    print("accuracy:", sum(correct) / (sum(correct) + sum(incorrect)))


if __name__ == "__main__":
    dataset, id2label, label2id, image_processor = get_dataset(PATH_TO_DATASET)
    labels = label2id.keys()
    model = setup_model(MODEL_PATH, id2label, label2id, labels)
    finetune_model(model, dataset, image_processor)
    validate_model(model, dataset["validation"])


Resolving data files:   0%|          | 0/699 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/99 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/202 [00:00<?, ?it/s]

Dataset loaded successfully.


Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded successfully.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.356800,0.124549,0.960396
2,0.086400,0.083572,0.970297
3,0.060000,0.081624,0.975248


incorrect prediction... predicted: non-cancerous expected: cancerous

 cancerous
right: 77  wrong: 1
accuracy: 0.9871794871794872

 non-cancerous
right: 21  wrong: 0
accuracy: 1.0

overall
right: 98 wrong: 1
accuracy: 0.98989898989899
